<img src="https://github.com/IKNL/guidelines/blob/master/resources/logos/iknl_nl.png?raw=true" width=150 align="right">
<br>
<b><font size="8"> DYNAMO Input Files - Automated </font></b>

<br>
<br>
<br>

### Importing libraries and definining functions

In [2]:
import xml.etree.ElementTree as ET
from datetime import datetime
import pandas as pd

In [3]:
def add_scenario(tree):
    # scenario_elements = [elem.tag for elem in tree.find('scenarios').find('scenario').iter()][1:]
    scenario_elements = ['uniquename','successRate','targetMinAge','targetMaxAge','targetSex','transfilename','prevfilename']
    scenarios = tree.find('scenarios')
    new_scenario = ET.SubElement(scenarios,"scenario")
    for scenario_element in scenario_elements:
        ET.SubElement(new_scenario, scenario_element)
    return(tree, new_scenario)

In [4]:
def update_scenario(scenario, 
                    uniquename=False, successRate=False, targetMinAge=False, 
                    targetMaxAge=False, targetSex=False, transfilename=False, 
                    prevfilename=False):
    
    if uniquename: scenario.find('uniquename').text = uniquename
    if successRate: scenario.find('successRate').text = successRate
    if targetMinAge: scenario.find('targetMinAge').text = targetMinAge
    if targetMaxAge: scenario.find('targetMaxAge').text = targetMaxAge
    if targetSex: scenario.find('targetSex').text = targetSex
    if transfilename: scenario.find('transfilename').text = transfilename
    if prevfilename: scenario.find('prevfilename').text = prevfilename

    return(scenario)

In [5]:
def show_tree(tree):
    root = tree.getroot()
    for i, child in enumerate(root):
        print(i, child.tag, child.attrib)
        
        for i, sub_child in enumerate(child):
            print('---', i, sub_child.tag, sub_child.attrib)

            for i, sub_sub_child in enumerate(sub_child):
                print('------', i, sub_sub_child.tag, sub_sub_child.attrib, sub_sub_child.text)

In [6]:
def get_and_delete_default_scenario(tree):
    root = tree.getroot()

    default_scenario = tree.find('scenarios').find('scenario')
    defaults = dict()

    if default_scenario:
        defaults['default_uniquename'] = default_scenario.find('uniquename').text
        defaults['default_successRate'] = default_scenario.find('successRate').text
        defaults['default_targetMinAge'] = default_scenario.find('targetMinAge').text
        defaults['default_targetMaxAge'] = default_scenario.find('targetMaxAge').text
        defaults['default_targetSex'] = default_scenario.find('targetSex').text
        defaults['default_transfilename'] = default_scenario.find('transfilename').text
        defaults['default_prevfilename'] = default_scenario.find('prevfilename').text

        scenarios = root.find('scenarios')
        scenarios.remove(default_scenario)

    return(tree, default_scenario, defaults)

<br>
<br>
<br>

## 1. Scenario's varying on success_rate (i.e. % of population reached)

User settings:

In [519]:
# A configuration.xml file, with 0 or 1 scenario, is need as input for this code. 
# From this file the structure of the xml will be taken, and already filled in data will be taken as default.
# In the case of an already existing scenario the filled in data will be set as default as well.
file_ = '.\example_files\configuration.xml' # @USER: Provide the path of a configuration.xml file

n_scenarios = 10 # @USER: Set the desired number of scenarios
include_0 = True
include_100 = True

# Choose whether to use the default values or not
# True; Use default values (if available scenario), 
# False; Use the user defined settings below ".._set_by_user"
use_default = False # @USER: Choose True or False


# Or, set use_default to False, and you can set the values yourself: these will overwrite the defaults
targetMinAge_set_by_user = "0" # @USER: Set the minimum age for the scenario's
targetMaxAge_set_by_user = "95" # @USER: Set the maximum age for the scenario's
targetSex_set_by_user = "2" # @USER: Set the target sex for the scenario's
transfilename_set_by_user = "sim1Rose" # @USER: Provide the filename of the transitionmatrix
prevfilename_set_by_user = "Hoii" # @USER: Provide the filename of the risk factor prevalence


Success rates:

In [535]:
succes_rates = list(range(100//n_scenarios,100+1,100//n_scenarios))

if include_0:
    if 0 not in succes_rates: succes_rates = [0] + succes_rates
if include_100:
    if 100 not in succes_rates: succes_rates = succes_rates + [100]

n_scenarios = len(succes_rates)

succes_rates

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

Execution:

In [522]:
tree = ET.parse(file_)
tree, default_scenario, default_scenario_settings = get_and_delete_default_scenario(tree)

if default_scenario:
    print('Default scenario available.')
else:
    print('No default scenario available.')

if default_scenario and use_default:
    print('Default values are used for targetMinAge, targetMaxAge, targetSex, transfilename and prevfilename.')
else:
    print('User settings are used.')

for i_scenario in range(n_scenarios):
    tree, new_scenario = add_scenario(tree)

    if default_scenario and use_default:
        targetMinAge = default_scenario_settings['default_targetMinAge']
        targetMaxAge = default_scenario_settings['default_targetMaxAge']
        targetSex = default_scenario_settings['default_targetSex']
        transfilename = default_scenario_settings['default_transfilename']
        prevfilename = default_scenario_settings['default_prevfilename']

    else: # user settings will be used
        targetMinAge = targetMinAge_set_by_user
        targetMaxAge = targetMaxAge_set_by_user
        targetSex = targetSex_set_by_user
        transfilename = transfilename_set_by_user
        prevfilename = prevfilename_set_by_user
        
    new_scenario = update_scenario(new_scenario, 
                                   uniquename=f"Scenario{i_scenario}", 
                                   successRate=str(succes_rates[i_scenario]),
                                   targetMinAge=targetMinAge,
                                   targetMaxAge=targetMaxAge,
                                   targetSex=targetSex,
                                   transfilename=transfilename,
                                   prevfilename=prevfilename)
    
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
output_xml = f'configuration_varying_succesrates_{dt_string}.xml' 
tree.write(output_xml)

print(f'Created: {output_xml}')

No default scenario available.
User settings are used.
Created: configuration_varying_succesrates_20240716_122334.xml


<br>
<br>
<br>

## 2. Different configurations for age groups with scenario's varying on success_rate (i.e. % of population reached)

User settings:

In [537]:
# A configuration.xml file, with 0 or 1 scenario, is need as input for this code. 
# From this file the structure of the xml will be taken, and already filled in data will be taken as default.
# In the case of an already existing scenario the filled in data will be set as default as well.
file_ = '.\example_files\configuration2.xml' # @USER: Provide the path of a configuration.xml file

n_scenarios = 10 # @USER: Set the desired number of scenarios
include_0 = True
include_100 = True

# A configuration.xml file will be created per age group. The user can define age groups. Example:
# age_groups = [ 
#     (0,49),
#     (50,100)
#     ]
# This example contains two age groups: 
# group one ranges from ages 0 to 49, 
# and group two from 50 to 100 years old.
age_groups = [ # @USER: Define desired age groups. List of tuples (age groups) with (minAge, maxAge) per age group.
    (0,18),
    (70,95)
    ] 


# Choose whether to use the default values or not
# True; Use default values (if available scenario), 
# False; Use the user defined settings below ".._set_by_user"
use_default = False # @USER: Choose True or False


# Or, set use_default to False, and you can set the values yourself: these will overwrite the defaults
targetSex_set_by_user = "2" # @USER: Set the target sex for the scenario's. 0: male, 1: female, 2: both
transfilename_set_by_user = "sim1Rose" # @USER: Provide the filename of the transition
prevfilename_set_by_user = "Hoii" # @USER: Provide the filename of the risk factor prevalence

Success rates:

In [533]:
succes_rates = list(range(100//n_scenarios,100+1,100//n_scenarios))

if include_0:
    if 0 not in succes_rates: succes_rates = [0] + succes_rates
if include_100:
    if 100 not in succes_rates: succes_rates = succes_rates + [100]

n_scenarios = len(succes_rates)

print(f"succes_rates: {succes_rates}")

succes_rates: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


Execution:

In [534]:
tree = ET.parse(file_)
tree, default_scenario, default_scenario_settings = get_and_delete_default_scenario(tree)

if default_scenario:
    print('Default scenario available.')
else:
    print('No default scenario available.')

if default_scenario and use_default:
    print('Default values are used for targetSex, transfilename and prevfilename.')
else:
    print('User settings are used.')
    

for age_group in age_groups:
    
    tree = ET.parse(file_)
    tree, default_scenario, default_scenario_settings = get_and_delete_default_scenario(tree)

    for i_scenario in range(n_scenarios):
        tree, new_scenario = add_scenario(tree)

        if default_scenario and use_default:
            targetSex = default_scenario_settings['default_targetSex']
            transfilename = default_scenario_settings['default_transfilename']
            prevfilename = default_scenario_settings['default_prevfilename']

        else: # user settings will be used
            targetSex = targetSex_set_by_user
            transfilename = transfilename_set_by_user
            prevfilename = prevfilename_set_by_user
            
        new_scenario = update_scenario(new_scenario, 
                                    uniquename=f"Scenario{i_scenario}", 
                                    successRate=str(succes_rates[i_scenario]),
                                    targetMinAge=str(age_group[0]),
                                    targetMaxAge=str(age_group[1]),
                                    targetSex=targetSex,
                                    transfilename=transfilename,
                                    prevfilename=prevfilename)
        
    now = datetime.now()
    dt_string = now.strftime("%Y%m%d_%H%M%S")
    output_xml = f"configuration_age_group_{str(age_group[0])}_to_{str(age_group[1])}_{dt_string}.xml"
    tree.write(output_xml)

    print(f'Created: {output_xml}')

Default scenario available.
User settings are used.
Created: configuration_age_group_0_to_18_20240716_122548.xml
Created: configuration_age_group_70_to_95_20240716_122548.xml


<br>
<br>
<br>

## 3. Create custom scenario's

User settings:

In [571]:
# The user has many options for custom settings. 
# The cell below this one will print the chosen user settings, always check if everything is according to what you expected.

# A configuration.xml file, with 0 or 1 scenario, is need as input for this code. 
# From this file the structure of the xml will be taken, and already filled in data will be taken as default.
# In the case of an already existing scenario the filled in data will be set as default as well.
file_ = '.\example_files\configuration2.xml' # @USER: Provide the path of a configuration.xml file

n_scenarios = 5 # @USER: Set the desired number of scenarios
include_0 = True
include_100 = True

# The user could provide names for the scenario's. This should be a list with strings with the same lenght as the number of scenarios.
# If you want to use the scenario names "Scenario0", "Scenario1", .. etc. use: 
# [f"Scenario{i}" for i in range(n_scenarios)] 
scenario_names = [f"Scenario{i}" for i in range(n_scenarios)] # @USER: Provide scenario names


# There are different options for the success_rate_type:
# "constant", "varying", or "custom"
# constant: In all scenario's the success rate is constant, the user has to set "success_rate_value"
# varying: The success rates are varied over the scenario's, the user has to set "succes_rates_bins", this can also be set to n_scenarios
# custom: The user can set custom rates (0 to 100), provide a list with as many items as n_scenarios, the user has to set "succes_rates_custom"
success_rate_type = "varying" # @USER: Set the success rate type to either "constant", "varying", or "custom"

if success_rate_type == "constant":
    success_rate_value = 100 # @USER: Set this when using success_rate_type = "constant"
    succes_rates = [success_rate_value]*n_scenarios 
elif success_rate_type == "varying":
    succes_rates_bins = n_scenarios # @USER: Set this when using success_rate_type = "varying"
    succes_rates = list(range(100//succes_rates_bins,100+1,100//succes_rates_bins))
    if include_0:
        if 0 not in succes_rates: succes_rates = [0] + succes_rates
    if include_100:
        if 100 not in succes_rates: succes_rates = succes_rates + [100]
    n_scenarios = len(succes_rates)

elif success_rate_type == "custom":
    succes_rates_custom = [10,20,30,40,50] # @USER: Set this when using success_rate_type = "custom"
    succes_rates = succes_rates_custom


targetSex = 2 # @USER: Set the target sex for the scenario's, where 0: male, 1: female, 2: both
targetSexs = [targetSex]*n_scenarios

transfilename = "fileX" # @USER: Provide the filename of the transitionmatrix
transfilenames = [transfilename]*n_scenarios

prevfilename = "filenameY" # @USER: Provide the filename of the risk factor prevalence
prevfilenames = [prevfilename]*n_scenarios 


# There are two options for the target ages (target_ages_type):
# "constant" or "groups"
# constant: In all scenario's the min and max age are constant, the user has to set "target_min_age" and "target_max_age"
# groups: Different configuration files will be made for different age groups, the user has to set "age_groups"
target_ages_type = "constant" # @USER: Set the taget ages type to either "constant" or "groups"

if target_ages_type == "constant":
    target_min_age = 0 # @USER: Set this when using target_ages_type = "constant"
    target_max_age = 95 # @USER: Set this when using target_ages_type = "groups"
    age_groups = [(target_min_age, target_max_age)]
elif target_ages_type == "groups":
    age_groups = [ # @USER: Define desired age groups. List of tuples (age groups) with (minAge, maxAge) per age group.
        (0,17),
        (18,25),
        (71,95)
        ]
    target_min_ages = [g[0] for g in age_groups] 
    target_max_ages = [g[1] for g in age_groups]


In [572]:
print(f"file: {file_}")
print(f"n_scenarios: {n_scenarios}")
print(f"scenario_names: {scenario_names}")
print(f"success_rate_type: {success_rate_type}")
print(f"succes_rates: {succes_rates}")
print(f"targetSexs: {targetSexs}")
print(f"transfilenames: {transfilenames}")
print(f"prevfilenames: {prevfilenames}")
print(f"target_ages_type: {target_ages_type}")
print(f"age_group(s): {age_groups}")

file: .\example_files\configuration2.xml
n_scenarios: 5
scenario_names: ['Scenario0', 'Scenario1', 'Scenario2', 'Scenario3', 'Scenario4']
success_rate_type: varying
succes_rates: [20, 40, 60, 80, 100]
targetSexs: [2, 2, 2, 2, 2]
transfilenames: ['fileX', 'fileX', 'fileX', 'fileX', 'fileX']
prevfilenames: ['filenameY', 'filenameY', 'filenameY', 'filenameY', 'filenameY']
target_ages_type: constant
age_group(s): [(0, 95)]


Execution:

In [573]:
for age_group in age_groups:
    
    tree = ET.parse(file_)
    tree, default_scenario, default_scenario_settings = get_and_delete_default_scenario(tree)

    for i_scenario in range(n_scenarios):
        tree, new_scenario = add_scenario(tree)
            
        new_scenario = update_scenario(new_scenario, 
                                    uniquename=str(scenario_names[i_scenario]),
                                    successRate=str(succes_rates[i_scenario]),
                                    targetMinAge=str(age_group[0]),
                                    targetMaxAge=str(age_group[1]),
                                    targetSex=str(targetSexs[i_scenario]),
                                    transfilename=str(transfilenames[i_scenario]),
                                    prevfilename=str(prevfilenames[i_scenario]))
        
    now = datetime.now()
    dt_string = now.strftime("%Y%m%d_%H%M%S")
    output_xml = f"custom_configuration_age_group_{str(age_group[0])}_to_{str(age_group[1])}_{dt_string}.xml"
    tree.write(output_xml)

    print(f'Created: {output_xml}')

Created: custom_configuration_age_group_0_to_95_20240716_141429.xml


<br>
<br>
<br>

## Create transitionmatrix

User settings:

In [7]:
start_age = 0 # @USER: Set the start age
stop_age = 95 # @USER: Set the stop age
n_categories = 3 # @USER: Set the number of categories

# The user should define the transition chances. This should be a list of tuples stating 
# the transition chance from a certain category to a certain category. The syntax is as follows: 
# [(FROM CATEGORY, TO CATEGORY, TRANSITION CHANCE in percentages),]
# 
# Example:
# transition_chances = [
#     (1,1,100),
#     (2,2,100),
#     (3,2,50),
#     (3,3,50)
#     ]
# In this example people from category 1 have a chance of 100% to stay category 1,
# people from category 2 stay category 2 as well, and people in category 3 have a 50% 
# chance to stay in category 3, and a 50% chance to change to category 2. 
# All other transitions are set to 0.
transition_chances = [ # @USER: Define transition chances
    (1,1,100),
    (2,2,100),
    (3,2,50),
    (3,3,50)
    ] 

Execution:

In [8]:
# Create a transitionmatrix
def flatten(xss):
    return [x for xs in xss for x in xs]

age_range = list(range(start_age,stop_age+1))
gender = ['0', '1'] # 0: male, 1: female
categories = list(range(1, n_categories+1))

length_of_frame = len(age_range)*len(gender)*(n_categories*n_categories)

age_column = age_range*len(gender)*(n_categories*n_categories)
gender_column = sorted(gender*len(age_range)*(n_categories*n_categories))
categories_column = flatten([[category]*len(age_range)*n_categories for category in categories]*len(gender))
transition_category_column = flatten([[category]*len(age_range) for category in categories]*n_categories*len(gender))

data = pd.DataFrame({'age': age_column, 
                     'gender': gender_column, 
                     'category': categories_column, 
                     'transition_category': transition_category_column})

data['transition_chance'] = 0
for transition in transition_chances:
    data.loc[((data.category==transition[0]) & (data.transition_category==transition[1])), 'transition_chance'] = transition[2]

pivotted_data = pd.pivot_table(data=data, index="age", columns=['gender', 'category', 'transition_category'], values='transition_chance').astype('int')

tmp = pivotted_data.columns.to_frame().T
tmp.loc[len(tmp)] = ''
tmp.rename(index={3:'age', 'category':'FROM', 'transition_category': 'TO'}, inplace=True)
pivotted_data = pd.concat([tmp, pivotted_data])
pivotted_data.columns = range(1,len(pivotted_data.columns)+1)

pivotted_data # Show pivotted data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
gender,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
FROM,1,1,1,2,2,2,3,3,3,1,1,1,2,2,2,3,3,3
TO,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3
age,,,,,,,,,,,,,,,,,,
0,100,0,0,0,100,0,0,50,50,100,0,0,0,100,0,0,50,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,100,0,0,0,100,0,0,50,50,100,0,0,0,100,0,0,50,50
92,100,0,0,0,100,0,0,50,50,100,0,0,0,100,0,0,50,50
93,100,0,0,0,100,0,0,50,50,100,0,0,0,100,0,0,50,50
94,100,0,0,0,100,0,0,50,50,100,0,0,0,100,0,0,50,50


In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

age_range = list(range(start_age,stop_age+1))
gender = ['0', '1'] # 0: male, 1: female
categories = list(range(1, n_categories+1))

length_of_frame = len(age_range)*len(gender)*(n_categories*n_categories)

age_column = age_range*len(gender)*(n_categories*n_categories)
gender_column = sorted(gender*len(age_range)*(n_categories*n_categories))
categories_column = flatten([[category]*len(age_range)*n_categories for category in categories]*len(gender))
transition_category_column = flatten([[category]*len(age_range) for category in categories]*n_categories*len(gender))

data = pd.DataFrame({'age': age_column, 
                     'gender': gender_column, 
                     'category': categories_column, 
                     'transition_category': transition_category_column})

In [ ]:
def create_matrix(start_age, stop_age, n_categories, transition_chances):
    # Create a transitionmatrix

    age_range = list(range(start_age,stop_age+1))
    gender = ['0', '1'] # 0: male, 1: female
    categories = list(range(1, n_categories+1))

    length_of_frame = len(age_range)*len(gender)*(n_categories*n_categories)

    age_column = age_range*len(gender)*(n_categories*n_categories)
    gender_column = sorted(gender*len(age_range)*(n_categories*n_categories))
    categories_column = flatten([[category]*len(age_range)*n_categories for category in categories]*len(gender))
    transition_category_column = flatten([[category]*len(age_range) for category in categories]*n_categories*len(gender))

    data = pd.DataFrame({'age': age_column, 
                        'gender': gender_column, 
                        'category': categories_column, 
                        'transition_category': transition_category_column})

    data['transition_chance'] = 0
    for transition in transition_chances:
        data.loc[((data.category==transition[0]) & (data.transition_category==transition[1])), 'transition_chance'] = transition[2]

    return(data)

In [539]:
# Create an xml file with transitionmatrix
root = ET.Element('transitionmatrix')

for idx,row in data.iterrows():
    transition = ET.SubElement(root, 'transition')
    age = ET.SubElement(transition, 'age')
    sex = ET.SubElement(transition, 'sex')
    from_ = ET.SubElement(transition, 'from')
    to = ET.SubElement(transition, 'to')
    percent = ET.SubElement(transition, 'percent')
    age.text = str(row.age)
    sex.text = str(row.gender)
    from_.text = str(row.category)
    to.text = str(row.transition_category)
    percent.text = str(row.transition_chance)

tree = ET.ElementTree(root)

now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
output_xml = f"transition_matrix_{dt_string}.xml"
tree.write(output_xml)
print(f'Created: {output_xml}')

Created: transition_matrix_20240716_130110.xml


<br>
<br>
<br>

## Archive

In [461]:
# ET.dump(new_scenario)
# pivotted_data.to_excel('even_proberen.xlsx')

# tree, new_scenario = add_scenario(tree)
# new_scenario = update_scenario(new_scenario, 
#                                uniquename="Scenario1", 
#                                successRate="0", 
#                                targetMinAge="0", 
#                                targetMaxAge="95", 
#                                targetSex="2", 
#                                transfilename='sim1Rose', 
#                                prevfilename='Name')